In [6]:
import os
import cv2
import numpy as np
from imutils import paths
import tensorflow as tf
from tensorflow import keras

In [7]:
from sklearn.preprocessing import LabelBinarizer

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
# from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [23]:
datapath = "C:/Users/Dell/Desktop/videoclassifier/data"
outputmodel = "C:/Users/Dell/Desktop/videoclassifier/video_classification_model/videoClassificationModel.keras"
outputlabelbinarizer = "C:/Users/Dell/Desktop/videoclassifier/model/videoClassificationBinarizer"
epoch = 25

In [11]:
Sports_Labels = set(['boxing', 'swimming','table_tennis'])
print("Images are being loaded...")
pathToImages = list(paths.list_images(datapath))
data = []
labels = []

for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in Sports_Labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image,(244,244))
    data.append(image)
    labels.append(label)

Images are being loaded...


In [12]:
data = np.array(data)
labels = np.array(labels)
#hot encoded values as 0,1,2
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [13]:
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

In [14]:
# Define the mean values for normalization
mean = np.array([123.68, 116.779, 103.939], dtype="float32")

# Define a function to normalize images
def preprocess_image(img):
    return img - mean

# Initialize ImageDataGenerator for training with augmentations
traininAugmentation = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",
    preprocessing_function=preprocess_image  # Apply normalization
)

# Initialize ImageDataGenerator for validation without augmentations
validationAugmentation = ImageDataGenerator(
    preprocessing_function=preprocess_image  # Apply normalization
)


In [15]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

In [16]:
baseModel = ResNet50(weights="imagenet", include_top= False, input_tensor=Input(shape=(244,244,3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

for basemodelLayers in baseModel.layers:
    basemodelLayers.trainabble = False

In [17]:
from tensorflow.keras.optimizers import SGD

In [18]:
opt = SGD(learning_rate=0.0001, momentum=0.9, decay=1e-4/epoch)

C:\Users\Dell\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [19]:
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [20]:
History = model.fit(
    traininAugmentation.flow(X_train, Y_train, batch_size=32),
    steps_per_epoch=len(X_train)//32,
    validation_data=validationAugmentation.flow(X_test, Y_test),
    validation_steps=len(X_test)//32,
    epochs=epoch
)

Epoch 1/25


C:\Users\Dell\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


49/49 ━━━━━━━━━━━━━━━━━━━━ 734s 13s/step - accuracy: 0.3928 - loss: 1.3790 - val_accuracy: 0.8086 - val_loss: 0.5351
Epoch 2/25
 1/49 ━━━━━━━━━━━━━━━━━━━━ 9:16 12s/step - accuracy: 0.6562 - loss: 0.8750

C:\Users\Dell\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


49/49 ━━━━━━━━━━━━━━━━━━━━ 14s 42ms/step - accuracy: 0.6562 - loss: 0.8750 - val_accuracy: 0.9091 - val_loss: 0.4293
Epoch 3/25
49/49 ━━━━━━━━━━━━━━━━━━━━ 642s 13s/step - accuracy: 0.6987 - loss: 0.7156 - val_accuracy: 0.9062 - val_loss: 0.3007
Epoch 4/25
49/49 ━━━━━━━━━━━━━━━━━━━━ 14s 33ms/step - accuracy: 0.8750 - loss: 0.3979 - val_accuracy: 0.8182 - val_loss: 0.4489
Epoch 5/25
49/49 ━━━━━━━━━━━━━━━━━━━━ 2519s 52s/step - accuracy: 0.8127 - loss: 0.5000 - val_accuracy: 0.9316 - val_loss: 0.2165
Epoch 6/25
49/49 ━━━━━━━━━━━━━━━━━━━━ 14s 33ms/step - accuracy: 0.8438 - loss: 0.4264 - val_accuracy: 0.9091 - val_loss: 0.2577
Epoch 7/25
49/49 ━━━━━━━━━━━━━━━━━━━━ 637s 13s/step - accuracy: 0.8600 - loss: 0.3888 - val_accuracy: 0.9277 - val_loss: 0.1917
Epoch 8/25
49/49 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.8750 - loss: 0.3574 - val_accuracy: 0.8182 - val_loss: 0.4402
Epoch 9/25
49/49 ━━━━━━━━━━━━━━━━━━━━ 685s 14s/step - accuracy: 0.8847 - loss: 0.3045 - val_accuracy: 0.9473 - val

In [21]:
import pickle

In [24]:
os.makedirs(os.path.dirname(outputmodel), exist_ok=True)

# Save the model
try:
    model.save(outputmodel)
    print(f"Model saved successfully at: {outputmodel}")
except Exception as e:
    print(f"Error saving model: {e}")

# Define the path for the label binarizer
binarizer_path = r"C:/Users/Dell/Desktop/videoclassifier/video_classification_model/videoClassificationBinarizer.pickle"

# Create the directory for the binarizer if it doesn't exist
os.makedirs(os.path.dirname(binarizer_path), exist_ok=True)

# Save the label binarizer
try:
    with open(binarizer_path, "wb") as lbinarizer:
        pickle.dump(lb, lbinarizer)
        print(f"Binarizer saved successfully at: {binarizer_path}")
except Exception as e:
    print(f"Error saving binarizer: {e}")

Model saved successfully at: C:/Users/Dell/Desktop/videoclassifier/video_classification_model/videoClassificationModel.keras
Binarizer saved successfully at: C:/Users/Dell/Desktop/videoclassifier/video_classification_model/videoClassificationBinarizer.pickle
